In [1]:
import os
from google.colab import drive

if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

In [2]:
!cp -r /content/drive/MyDrive/.ssh/ ~/
!ls ~/.ssh/ -a
!ssh -T git@github.com
!git clone git@github.com:Yichuan0712/CoTable.git

.  ..  github_id_rsa  hellbender_id_rsa  id_rsa  id_rsa.pub  known_hosts
Hi Yichuan0712! You've successfully authenticated, but GitHub does not provide shell access.
Cloning into 'CoTable'...
remote: Enumerating objects: 638, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 638 (delta 102), reused 96 (delta 49), pack-reused 486 (from 1)
Receiving objects: 100% (638/638), 1.23 MiB | 6.13 MiB/s, done.
Resolving deltas: 100% (367/367), done.


In [3]:
!cp /content/drive/MyDrive/OSU/tabula_data.env /content/.env
# !cp /content/drive/MyDrive/OSU/yichuan_gemini.env /content/.env

In [4]:
!pip install python-dotenv -q
!pip install fake_useragent -q
!pip install shortuuid -q
!pip install ratelimit -q
!pip install langchain-openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.8 MB/s eta 0:00:00


In [5]:
import os
os.chdir('/content/CoTable')

In [6]:
!git pull origin main

From github.com:Yichuan0712/CoTable
 * branch            main       -> FETCH_HEAD
Already up to date.


In [7]:
from llm_utils import *
import google.generativeai as genai
# Important for Colab: Manually configure GEMINI_15_API_KEY to enable Gemini API usage
if "GEMINI_15_API_KEY" in os.environ:
    genai.configure(api_key=os.environ.get("GEMINI_15_API_KEY", None))

In [8]:
STOP

NameError: name 'STOP' is not defined

In [9]:
from table_utils import *
from llm_utils import *
from steps.s_pk_delete_individual import *
from steps.s_pk_align_parameter import *
# from steps.s_pk_split_by_rows import *
# from steps.s_pk_split_by_cols import *
from steps.s_pk_extract_drug_info import *
from steps.s_pk_extract_patient_info import *
from steps.s_pk_get_col_mapping import *
from steps.s_pk_get_parameter_type_and_unit import *
from steps.s_pk_match_drug_info import *
from steps.s_pk_match_patient_info import *

In [14]:
llm = "chatgpt_4o"
# llm = "gemini_15_pro"

pmid = "34114632"
tableid = "2"
filepath = f'/content/CoTable/data/html/pk_summary_250218/{pmid}/{tableid}.html'
md_table = deduplicate_headers(fill_empty_headers(remove_empty_col_row(stack_md_table_headers(html_table_to_markdown(get_html_content_from_file(filepath))))))

json_filepath = f'/content/CoTable/data/html/pk_summary_250218/{pmid}/{tableid}.json'
caption, footnote = get_caption_and_footnote_from_file(json_filepath)
caption = caption + footnote

print(display_md_table(md_table))
print()
print('Caption:', caption)
print("\n"*2)

col: | ".-> PK parameters." | "Dose range 1 (5–15 mg/kg/12 h).->Historical data." | "Dose range 1 (5–15 mg/kg/12 h).->Current data." | "Dose range 1 (5–15 mg/kg/12 h).->pvalue." | "Dose range 2 (15–25 mg/kg/12 h).->Historical data." | "Dose range 2 (15–25 mg/kg/12 h).->Current data." | "Dose range 2 (15–25 mg/kg/12 h).->pvalue." | "Dose range 3 (25–35 mg/kg/12 h).->Historical data." | "Dose range 3 (25–35 mg/kg/12 h).->Current data." | "Dose range 3 (25–35 mg/kg/12 h). ->pvalue." |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
row 0: | Cmax(µg/mL) | 24.8 ± 8.3 | 19.19 ± 4.12 | 0.07 | 57.1 ± 14.9 | 35.12 ± 10.54 | 0.001 | 73.2 ± 19.2 | 36.11 (27.58–44.64) | – |
row 1: | AUC0–12(h*µg /mL) | 145 ± 44 | 167.0 ± 45.6 | 0.26 | 322 ± 71 | 316.5 ± 108.4 | 0.88 | 433 ± 94 | 290.9 (176.14–405.59) | – |
row 2: | Ctrough(µg/mL) | 8.4 ± 3.8 | 9.99 ± 3.86 | 0.34 | 15.6 ± 5.3 | 19.25 ± 8.48 | 0.22 | 20.6 ± 5.8 | 13.03 (2.98–23.07) | – |
row 3: | Tmax(h) | 0.5 (0.25–3.0) | 1.5 (1.5–2.5

In [32]:
md_table_drug, res_drug, content_drug, usage_drug, truncated_drug = s_pk_extract_drug_info(md_table, caption, llm)
print("\n"*2)

In [34]:
markdown_to_dataframe(md_table_drug).shape[0]

1

In [ ]:
md_table_patient, res_patient, content_patient, usage_patient, truncated_patient = s_pk_extract_patient_info(md_table, caption, llm)
print("\n"*2)

In [15]:
md_table_summary, res_summary, content_summary, usage_summary, truncated_summary = s_pk_delete_individual(md_table, llm)
print("\n"*2)

In [16]:
md_table_aligned, res_aligned, content_aligned, usage_aligned, truncated_aligned = s_pk_align_parameter(md_table_summary, llm)
print("\n"*2)

In [17]:
col_mapping, res_mapping, content_mapping, usage_mapping, truncated_mapping = s_pk_get_col_mapping(md_table_aligned, llm)
print("\n"*2)

20853 To categorize each column header correctly, I need to carefully consider the information presented in the table headers and match them to one of the predefined categories. Here's my thought process:

1. **Parameter Type**: This group includes columns indicating the type of pharmacokinetic parameter being measured, such as Cmax, AUC0–12, Ctrough, and Tmax.
2. **Parameter Unit**: This group includes columns that specify the unit of measure for each parameter type. Given the table header labels, it's clear that unit details are embedded within the parameter names, so there are no separate parameter unit columns.
3. **Parameter Value**: This group includes columns that contain the actual numerical values of the pharmacokinetic parameters, be it historical or current data.
4. **P Value**: This group includes columns that represent the statistical significance of the differences between historical and current data.
5. **Uncategorized**: This group would include any columns that don't f

In [31]:
import re
import ast
from table_utils import *
from llm_utils import *
from operations.f_split_by_cols import *


# def s_pk_split_by_cols_prompt(md_table, col_mapping):
#     mapping_str = ""
#     for k, v in col_mapping.items():
#       mapping_str += f""""{k}" """ + "is categorized as" + f""" "{v}," \n"""
#     # print(mapping_str)
#     parameter_type_count = list(col_mapping.values()).count("Parameter type")
#     parameter_pvalue_count = list(col_mapping.values()).count("P value")
#     situation_str = ""
#     if parameter_pvalue_count > 1 and parameter_type_count <= 1:
#       situation_str += f"""因为有多个被分类为 "P value" 的列,"""
#     elif parameter_type_count > 1 and parameter_pvalue_count <= 1:
#       situation_str += f"""因为有多个被分类为 "Parameter type" 的列,"""
#     elif parameter_type_count > 1 and parameter_pvalue_count > 1:
#       situation_str += f"""因为有多个被分类为 "Parameter type" 和 "P value" 的列,"""
#     else:
#       pass  # it will not get here

#     return f"""
# There is now a table related to pharmacokinetics (PK).
# {display_md_table(md_table)}
# 这个表格有很多列, 其中:
# {mapping_str}
# 这个表格是可以拆分成多个子表的, {situation_str}
# Carefully examine the table and follow these steps:
# (1) Review all columns, 检查各个列之间的关系, group the columns accordingly.
# (2) 在每个分组中, 必定包含一个被分类为 "Parameter type" 的列, 最多包含一个被分类为 "P value" 的列
# Use the function below (do not over-split based on column categories):
# f_split_by_cols(col_groups)
# Replace col_groups with column names in the following format:
# col_groups = [["ColumnA", "ColumnB", "ColumnC", "ColumnG"], ["ColumnA", "ColumnD", "ColumnE", "ColumnF", "ColumnG"]] (example)
# When returning this, enclose the function call in double angle brackets, like this:
# <<f_split_by_cols([["ColumnA", "ColumnB", "ColumnC", "ColumnG"], ["ColumnA", "ColumnD", "ColumnE", "ColumnF", "ColumnG"]])>>
# """

def s_pk_split_by_cols_prompt(md_table, col_mapping):
    """
    Generates a structured prompt for splitting a pharmacokinetics (PK) table into sub-tables
    based on column classifications.

    Args:
        md_table (str): Markdown representation of the table.
        col_mapping (dict): Dictionary mapping column names to their respective categories.

    Returns:
        str: A formatted prompt guiding the splitting process.
    """
    mapping_str = "\n".join(f'"{k}" is categorized as "{v},"' for k, v in col_mapping.items())

    # Count occurrences of specific categories
    parameter_type_count = sum(1 for v in col_mapping.values() if v == "Parameter type")
    parameter_pvalue_count = sum(1 for v in col_mapping.values() if v == "P value")

    # Identify the situation based on category counts
    if parameter_pvalue_count > 1 and parameter_type_count <= 1:
        situation_str = "because there are multiple columns categorized as \"P value\","
    elif parameter_type_count > 1 and parameter_pvalue_count <= 1:
        situation_str = "because there are multiple columns categorized as \"Parameter type\","
    elif parameter_type_count > 1 and parameter_pvalue_count > 1:
        situation_str = "because there are multiple columns categorized as both \"Parameter type\" and \"P value\","
    else:
        situation_str = ""

    return f"""
There is a table related to pharmacokinetics (PK):
{display_md_table(md_table)}

This table contains multiple columns, categorized as follows:
{mapping_str}

This table can be split into multiple sub-tables {situation_str}.
Please follow these steps:
  (1) Carefully review all columns and analyze their relationships to determine logical groupings.
  (2) Ensure that each group contains exactly one 'Parameter type' column and at most one 'P value' column.

Return the results as a list of lists, where each inner list represents a sub-table with its included columns.
Enclose the final list within double angle brackets (<< >>) like this:
<<[["ColumnA", "ColumnB", "ColumnC", "ColumnG"], ["ColumnA", "ColumnD", "ColumnE", "ColumnF", "ColumnG"]]>>
"""
# print(s_pk_split_by_cols_prompt(md_table_aligned, col_mapping))

def s_pk_split_by_cols_parse(content):
    content = content.replace('\n', '')

    matches = re.findall(r'<<.*?>>', content)
    match_angle = matches[-1] if matches else None

    if match_angle:
        match_list = match_angle[2:-2]
        match_list = ast.literal_eval(match_list)
        return match_list
    else:
        raise NotImplementedError

def s_pk_split_by_cols(md_table, col_mapping, model_name="gemini_15_pro"):
    msg = s_pk_split_by_cols_prompt(md_table, col_mapping)

    messages = [msg, ]
    question = "Do not give the final result immediately. First, explain your thought process, then provide the answer."

    res, content, usage, truncated = get_llm_response(messages, question, model=model_name)
    # print(display_md_table(md_table))
    print(usage, content)

    col_groups = s_pk_split_by_cols_parse(content)
    if col_groups is None:
        NotImplementedError
        # return_md_tables = [md_table, ]
    else:
        col_groups = [[fix_col_name(item, md_table) for item in group] for group in col_groups]
        df_table = f_split_by_cols(col_groups, markdown_to_dataframe(md_table))
        return_md_table_list = []
        for d in df_table:
            return_md_table_list.append(dataframe_to_markdown(d))

    for m in return_md_table_list:
        print(display_md_table(m))

    return return_md_table_list, res, content, usage, truncated, col_groups

s_pk_split_by_cols(md_table_aligned, col_mapping, model_name="chatgpt_4o")


16210 Certainly! Let's break down the process step-by-step to ensure a clear understanding of how to split the table into multiple sub-tables.

### Step-by-Step Thought Process

1. **Identify the Unique Columns**:
   - We have a single 'Parameter type' column: `".-> PK parameters."`.
   - We have three dose ranges, each with historical data, current data, and p-values.

2. **Group Columns by Dose Range**:
   - Dose range 1 (5–15 mg/kg/12 h): We have historical data, current data, and a p-value.
   - Dose range 2 (15–25 mg/kg/12 h): We have historical data, current data, and a p-value.
   - Dose range 3 (25–35 mg/kg/12 h): We have historical data, current data, and a p-value.

3. **Create Logical Groupings**:
   - Each sub-table should include the 'Parameter type' column along with the corresponding historical data, current data, and p-value for each dose range.

4. **Construct Sub-Tables**:
   - Sub-table 1: Columns related to Dose range 1.
   - Sub-table 2: Columns related to Dose ran

(['| .-> PK parameters. | Dose range 1 (5–15 mg/kg/12 h).->Historical data. | Dose range 1 (5–15 mg/kg/12 h).->Current data. | Dose range 1 (5–15 mg/kg/12 h).->pvalue. |\n| --- | --- | --- | --- |\n| Cmax(µg/mL) | 24.8\u2009±\u20098.3 | 19.19\u2009±\u20094.12 | 0.07 |\n| AUC0–12(h*µg\u2009/mL) | 145\u2009±\u200944 | 167.0\u2009±\u200945.6 | 0.26 |\n| Ctrough(µg/mL) | 8.4\u2009±\u20093.8 | 9.99\u2009±\u20093.86 | 0.34 |\n| Tmax(h) | 0.5 (0.25–3.0) | 1.5 (1.5–2.5) | – |',
  '| .-> PK parameters. | Dose range 2 (15–25 mg/kg/12 h).->Historical data. | Dose range 2 (15–25 mg/kg/12 h).->Current data. | Dose range 2 (15–25 mg/kg/12 h).->pvalue. |\n| --- | --- | --- | --- |\n| Cmax(µg/mL) | 57.1\u2009±\u200914.9 | 35.12\u2009±\u200910.54 | 0.001 |\n| AUC0–12(h*µg\u2009/mL) | 322\u2009±\u200971 | 316.5\u2009±\u2009108.4 | 0.88 |\n| Ctrough(µg/mL) | 15.6\u2009±\u20095.3 | 19.25\u2009±\u20098.48 | 0.22 |\n| Tmax(h) | 0.5 (0.5–3.0) | 2.5 (2.0–3.3) | – |',
  '| .-> PK parameters. | Dose range 3 (25

In [18]:
tuple_type_unit, res_type_unit, content_type_unit, usage_type_unit, truncated_type_unit = s_pk_get_parameter_type_and_unit(col_mapping, md_table_aligned, caption, llm)

11686 Sure, let's go through the thought process step by step to achieve the desired results:

### Step-by-Step Thought Process:

1. **Identify the PK Parameters and Units:**
   Looking at the column ".-> PK parameters.", it includes both the names of the pharmacokinetic (PK) parameters and their units. We need to split these into separate lists for parameter types and parameter units.

2. **Extract Parameter Types:**
   The parameter types from the table are the following:
   - Cmax
   - AUC0–12
   - Ctrough
   - Tmax

3. **Extract Parameter Units:**
   Correspondingly, the units for these parameters are:
   - µg/mL (for Cmax)
   - h*µg/mL (for AUC0–12)
   - µg/mL (for Ctrough)
   - h (for Tmax)

4. **Validating the Extractions:**
   We need to ensure that these units correspond well to the pharmacokinetic parameters and that none of the values in the column is a non-parameter entry like a drug name. In this case, all the values appear to be valid PK parameters.

5. **Construct the Li

In [ ]:
md_table_aligned_with_1_param_type_and_value_list = get_1_param_type_and_value_sub_md_table_list(col_mapping, md_table_aligned)
drug_match_list, res_match_drug, content_match_drug, usage_match_drug, truncated_match_drug = s_pk_match_drug_info(md_table_aligned, caption, md_table_aligned_with_1_param_type_and_value_list[1], md_table_drug)

In [ ]:
print(display_md_table(md_table_aligned_with_1_param_type_and_value_list[1]))

In [ ]:
drug_match_list, res_match_drug, content_match_patient, usage_match_patient, truncated_match_patient = s_pk_match_patient_info(md_table_aligned, caption, md_table_aligned_with_1_param_type_and_value_list[0], md_table_patient)

In [ ]:
def get_1_param_type_and_value_sub_md_table_list(col_mapping, md_table_aligned):
    pt_list = []
    pv_list = []
    for k, v in col_mapping.items():
        if v == "Parameter type":
            pt_list.append(k)
        elif v == "Parameter value" or "P value":
            pv_list.append(k)
    assert len(pt_list) == 1
    md_table_aligned_with_1_param_type_and_value_list = []
    for pv in pv_list:
        md_table_aligned_with_1_param_type_and_value_list.append(dataframe_to_markdown(markdown_to_dataframe(md_table_aligned).iloc[:][[pt_list[0], pv]].reset_index(drop=True)))
    return md_table_aligned_with_1_param_type_and_value_list

In [ ]:
import time
from table_utils import *
from llm_utils import *
from steps.s_pk_delete_individual import *
from steps.s_pk_align_parameter import *
from steps.s_pk_extract_drug_info import *
from steps.s_pk_extract_patient_info import *
from steps.s_pk_get_col_mapping import *
from steps.s_pk_get_parameter_type_and_unit import *
from steps.s_pk_match_drug_info import *
from steps.s_pk_match_patient_info import *


def run_with_retry(func, *args, max_retries=5, base_delay=10, **kwargs):
    delay = base_delay
    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                delay *= 2
            else:
                print("Max retries reached. Returning None.")
                return None

    # it will not get here
    return None


def p_pk_summary(md_table, description, llm="gemini_15_pro", max_retries=5, base_delay=10, use_color=True):
    """
    PK Summary Pipeline 250227
    Summarizes pharmacokinetic (PK) data from a given markdown table.

    :param md_table: The markdown representation of a SINGLE HTML table containing PK data.
    :param description: Additional contextual information, including captions, footnotes, etc.
    :param llm: The language model to use for processing ("gemini_15_pro" or "chatgpt_4o").
    :param max_retries: Maximum number of retries before failing.
    :param base_delay: Initial delay before retrying (in seconds), which doubles on each failure.
    :return:
    """
    if use_color:
        # COLOR_START = "\033[31m"  # red
        COLOR_START = "\033[32m"  # green
        # COLOR_START = "\033[33m"  # yellow
        COLOR_END = "\033[0m"

    step_list = []
    res_list = []
    content_list = []
    usage_list = []
    truncated_list = []

    """
    Step 1: Extract Drug Information
    """
    print("=" * 64)
    step_name = "Extract Drug Information"
    print(COLOR_START+step_name+COLOR_END)
    drug_info = run_with_retry(
        s_pk_extract_drug_info,
        md_table,
        description,
        llm,
        max_retries=max_retries,
        base_delay=base_delay,
    )
    if drug_info is None:
        return None
    md_table_drug, res_drug, content_drug, usage_drug, truncated_drug = drug_info
    step_list.append(step_name)
    res_list.append(res_drug)
    content_list.append(content_drug)
    usage_list.append(usage_drug)
    truncated_list.append(truncated_drug)
    print(COLOR_START+"Usage:"+COLOR_END, usage_drug)
    print(COLOR_START+"Result:\n"+COLOR_END, display_md_table(md_table_drug))
    print(COLOR_START+"Reasoning:\n"+COLOR_END, content_drug)
    print("\n"*1)
    """
    Step 2: Extract Population Information
    """
    print("=" * 64)
    step_name = "Extract Population Information"
    print(COLOR_START+step_name+COLOR_END)
    patient_info = run_with_retry(
        s_pk_extract_patient_info,
        md_table,
        description,
        llm,
        max_retries=max_retries,
        base_delay=base_delay,
    )
    if patient_info is None:
        return None
    md_table_patient, res_patient, content_patient, usage_patient, truncated_patient = patient_info
    step_list.append(step_name)
    res_list.append(res_patient)
    content_list.append(content_patient)
    usage_list.append(usage_patient)
    truncated_list.append(truncated_patient)
    print(COLOR_START+"Usage:"+COLOR_END, usage_patient)
    print(COLOR_START+"Result:\n"+COLOR_END, display_md_table(md_table_patient))
    print(COLOR_START+"Reasoning:\n"+COLOR_END, content_patient)
    print("\n"*1)
    return




In [ ]:
p_pk_summary(md_table, caption, llm="gemini_15_pro", max_retries=5, base_delay=10)


In [ ]:
# print("\033[31mThis is red text\033[0m")  # 红色
# print("\033[32mThis is green text\033[0m")  # 绿色
# print("\033[33mThis is yellow text\033[0m")  # 黄色
# print("\033[33m"+"test"+"\033[0m", "test")


In [ ]:
# md_table = \
# """| Parturient | Cord blood (ng/ml) | Maternal blood (ng/ml) | Collection time(min) | Cord blood/maternal blood |
# | --- | --- | --- | --- | --- |
# | 1 | 5.77 | 14.74 | 135 | 0.392 |
# | 2 | 6.82 | 7.95 | 426 | 0.858 |
# | 3 | 4.38 | 10.48 | 153 | 0.418 |
# | 4 | 8.42 | 9.60 | 300 | 0.878 |
# | 5 | 5.87 | 5.33 | 390 | 1.100 |
# | 6 | 5.78 | 9.87 | 120 | 0.586 |
# | 7 | 7.75 | 10.94 | 552 | 0.708 |
# | 8 | 9.45 | 10.35 | 207 | 0.913 |"""

# """| bus | car | rabbit | dog | cat |
# | --- | --- | --- | --- | --- |
# | 1 | 5.77 | 14.74 | 135 | 0.392 |
# | 2 | 6.82 | 7.95 | 426 | 0.858 |
# | 3 | 4.38 | 10.48 | 153 | 0.418 |
# | 4 | 8.42 | 9.60 | 300 | 0.878 |
# | 5 | 5.87 | 5.33 | 390 | 1.100 |
# | 6 | 5.78 | 9.87 | 120 | 0.586 |
# | 7 | 7.75 | 10.94 | 552 | 0.708 |
# | 8 | 9.45 | 10.35 | 207 | 0.913 |"""